## MobileNet
- Mobile net is a model which gives reasonably good imagenet classification accuracy and occupies very less space. (17 MB according to keras docs).

In [11]:
'''For preprocessing images'''
import numpy as np
from PIL import Image
import scipy
import matplotlib.pyplot as plt
import csv
import glob
'''For CNN'''
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import MobileNet
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
from matplotlib.pyplot import imshow

In [2]:
image_size = 256
train_data_size = 4000
test_data_size = 1000

In [3]:
def standardize(img):
    # padding
    longer_side = max(img.size)
    horizontal_padding = (longer_side - img.size[0]) / 2
    vertical_padding = (longer_side - img.size[1]) / 2
    img = img.crop(
        (
            -horizontal_padding,
            -vertical_padding,
            img.size[0] + horizontal_padding,
            img.size[1] + vertical_padding
        )
    )
    # resizing to standardized size
    img = img.resize([image_size,image_size],Image.ANTIALIAS) \
    # plt.imshow(img) # To see the image being standardized.
    
    # converting image to numpy array
    img.load()
    img = np.asarray(img, dtype="int32")
    return img

In [4]:
def function():
    for filename in glob.glob('input/subset_data/train/*.tif'):
        img =Image.open(filename)
        img = standardize(img)
        print(img.shape)
        return

In [5]:
function()

(256, 256, 3)


In [6]:
'''Loading data'''
def get_id_from_filename(filename):
    id = filename.split("/")[-1]
    id = id.split(".")[0]
    return id

In [7]:
def load_train():
    names = []
    # Change first number base on number of training examples
    X_train = np.empty((train_data_size,image_size,image_size,3), dtype="int32")
    Y_train = np.empty(shape=(train_data_size,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/train/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_train[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_train[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_train[names.index(name)] = np.array([0,1]) # means 1
    return X_train,Y_train

In [8]:
def load_test():
    names = []
    # Change first number base on number of training examples
    X_test = np.empty((test_data_size,image_size,image_size,3), dtype="int32")
    Y_test = np.empty(shape=(test_data_size,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/test_with_outputs/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_test[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_test[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_test[names.index(name)] = np.array([0,1]) # means 1
    return X_test,Y_test

In [9]:
X_train_orig,Y_train_orig = load_train()
X_test_orig,Y_test_orig = load_test()

# Normalizing for faster convergence
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = Y_train_orig
Y_test = Y_test_orig

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

# To check values inside.
# print(X_train)
# print(Y_train)
# print(X_test)
# print(Y_test)

X_train shape:  (4000, 256, 256, 3)
Y_train shape:  (4000, 2)
X_test shape:  (1000, 256, 256, 3)
Y_test shape:  (1000, 2)


In [15]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [16]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [18]:
# for layer in model.layers:
#     layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [20]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

In [ ]:
model.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 32, validation_data=(X_test,Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 44s 11ms/step - loss: 0.7417 - acc: 0.5752 - val_loss: 0.6728 - val_acc: 0.6070
Epoch 2/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6797 - acc: 0.5877 - val_loss: 0.7242 - val_acc: 0.6070
Epoch 3/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6791 - acc: 0.5875 - val_loss: 0.6710 - val_acc: 0.6070
Epoch 4/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6768 - acc: 0.5877 - val_loss: 0.6733 - val_acc: 0.6070
Epoch 5/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6731 - acc: 0.5907 - val_loss: 0.6759 - val_acc: 0.6070
Epoch 6/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6654 - acc: 0.5952 - val_loss: 0.8432 - val_acc: 0.6070
Epoch 7/20
4000/4000 [==============================] - 37s 9ms/step - loss: 0.6625 - acc: 0.